# Selenium Basic command

1. driver.get('url') : 페이지 접속
2. elem = driver.find_element_by_name('q') : 커서를 검색어 입력 부분에 위치시킴
    - find_by_class_name or find_by_xpath 등도 가능 (xpath는 html 검사 시 우클릭>xpath copy로 복사 가능
3. elem.clear() : 검색어 부분 입력내용 삭제
4. elem.send_keys('내용') : 내용 입력
5. elem.submit() : 입력한 내용 검색 실행
    - click() : 클릭
6. driver.close() : 페이지 닫음
7. aseert 'keyword' in driver.title or driver.page_source : 제목이나 내용에 keyword가 있는지 파악 - 없으면 assertion error 반환

In [22]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from urllib.parse import quote_plus

import time
from datetime import datetime
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
def instagram_login(id, pw):
    driver.get(url)
    driver.implicitly_wait(5)
    driver.find_element_by_name('username').send_keys(id)    # id 입력
    elem_pw = driver.find_element_by_name('password')    # pw 입력
    elem_pw.send_keys(pw)
    elem_pw.submit()
    
    driver.implicitly_wait(5)    # 파싱될 때까지 5초 기다림 (미리 완료되면 waiting 종료됨)
    driver.find_element_by_class_name('cmbtv').click()    # 비밀번호 저장하지 않음
    
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()    # 알림설정 무시

In [3]:
def main_search(keyword):
    search = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
    search.clear()
    search.send_keys(keyword)
    search_list1 = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]')
    search_list1.click()

In [4]:
SCROLL_PAUSE_TIME = 2.0
post_link = []
def scroll_page():
    while True:
        pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
        bsObj = bs(pageString, 'lxml')

        for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
            a_len = len(postline.select('a'))
            # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
            for post in range(a_len):   
                item = postline.select('a')[post]
                link = item.attrs['href']
                post_link.append(link)



        last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
        time.sleep(SCROLL_PAUSE_TIME)
        # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            #driver.implicitly_wait(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            else:
                last_height = new_height
                continue

In [14]:
SCROLL_PAUSE_TIME = 2.0
post_link = []

while True:
    pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
    bsObj = bs(pageString, 'lxml')

    for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
        a_len = len(postline.select('a'))
        # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
        for post in range(a_len):   
            item = postline.select('a')[post]
            link = item.attrs['href']
            post_link.append(link)



    last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
    time.sleep(SCROLL_PAUSE_TIME)
    # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
    #driver.implicitly_wait(SCROLL_PAUSE_TIME)
    # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

# 날짜 관계없이 게시물 내용 끝까지 수집

In [190]:


num_of_post = len(post_link)

print('총 {0}개의 게시글을 수집합니다.'.format(num_of_post))

id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []

for i in tqdm_notebook(range(num_of_post)):
    
    tag_list.append([])
    
    # 개별 링크 리스트 
    link_list.append("https://www.instagram.com" + post_link[i])   
    req = Request("https://www.instagram.com" + post_link[i], headers={'User-Agent': 'Mozila/5.0'})
    postpage = urlopen(req).read()
    
    post_body = bs(postpage, 'lxml', from_encoding='utf-8')
    post_core = post_body.find('meta', attrs={'property':"og:description"})
    contents = post_core['content']

    # 시간    
    posttxt = str(postpage)    
    timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
    date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))
    month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
    day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))
    
    # 좋아요
    try:
        likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
    except:
        likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
    like_list.append(likes)
    
    # 개별 계정
    if "@" and ")" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find(")")]
    elif "@" and ")" not in contents and "on Instagram" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
    else :
        personal_id = contents[1: contents.find(' posted on')]
    id_list.append(personal_id)
    
    '''    
    (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
    때문에 여러 형식별 id 추출 if문 수행
    '''
    
    # 해시태그
    for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
        hashtags = tag_content['content'].rstrip(',')
        tag_list[i].append(hashtags)


총 32개의 게시글을 수집합니다.


# 날짜옵션 추가

In [20]:
id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []


post_linkx = post_link[8:]
num_of_postx = len(post_linkx)
    
def date_based_crawling(std_date):
    date_string = std_date
    date =datetime.strptime(date_string, "%Y.%m.%d")
    time_tuple = date.timetuple()
    std_timestamp = time.mktime(time_tuple)
    # 기준 날짜를 지나면 크롤링이 멈추도록 작동
    
    for i in tqdm_notebook(range(num_of_postx)):
        req = Request("https://www.instagram.com" + post_linkx[i], headers={'User-Agent': 'Mozila/5.0'})
        postpage = urlopen(req).read()

        post_body = bs(postpage, 'lxml', from_encoding='utf-8')
        post_core = post_body.find('meta', attrs={'property':"og:description"})
        contents = post_core['content']

        # break할 시간 비교
        posttxt = str(postpage)    
        timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])

        if std_timestamp > timestamp:
            break    
        
        # 시간
        date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))
        month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
        day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))
        
        # 개별 링크 리스트 
        link_list.append("https://www.instagram.com" + post_linkx[i])   
        
        # 좋아요
        try:
            likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
        except:
            likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
        like_list.append(likes)

        # 개별 계정
        if "@" and ")" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find(")")]
        elif "@" and ")" not in contents and "on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
        else :
            personal_id = contents[1: contents.find(' posted on')]
        id_list.append(personal_id)

        '''    
        (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
        때문에 여러 형식별 id 추출 if문 수행
        '''

        # 해시태그
        tag_list.append([])
        for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
            hashtags = tag_content['content'].rstrip(',')
            tag_list[i].append(hashtags)


post_linkx = post_link[10:]
num_of_postx = len(post_linkx)

id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []

def date_based_crawling(std_date):        
    date_string = std_date
    date =datetime.strptime(date_string, "%Y.%m.%d")
    time_tuple = date.timetuple()
    std_timestamp = time.mktime(time_tuple)
    # 기준 날짜를 지나면 크롤링이 멈추도록 작동
    
    print('총 {0}개의 게시글 중 {1}까지의 데이터를 수집합니다.'.format(num_of_post, date))
    
    for i in tqdm_notebook(range(num_of_post)):
        req = Request("https://www.instagram.com" + post_linkx[i], headers={'User-Agent': 'Mozila/5.0'})
        postpage = urlopen(req).read()

        post_body = bs(postpage, 'lxml', from_encoding='utf-8')
        post_core = post_body.find('meta', attrs={'property':"og:description"})
        contents = post_core['content']

        # break할 시간 비교
        posttxt = str(postpage)    
        timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])

        if std_timestamp > timestamp:
            break    
            
        tag_list.append([])
        
        # 시간
        date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))
        month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
        day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))
        
        # 개별 링크 리스트 
        link_list.append("https://www.instagram.com" + post_linkx[i])   
        
        # 좋아요
        try:
            likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
        except:
            likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
        like_list.append(likes)

        # 개별 계정
        if "@" and ")" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find(")")]
        elif "@" and ")" not in contents and "on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
        else :
            personal_id = contents[1: contents.find(' posted on')]
        id_list.append(personal_id)

        '''    
        (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
        때문에 여러 형식별 id 추출 if문 수행
        '''

        # 해시태그
        for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
            hashtags = tag_content['content'].rstrip(',')
            tag_list[i].append(hashtags)

In [213]:
insta_dict = {'날짜':date_list,
              '월':month_list,
              '일':day_list,
              '계정':id_list,
             '좋아요':like_list,
             '해시태그':tag_list,

             '링크':link_list}
df = pd.DataFrame(insta_dict)
df

,날짜,월,일,계정,좋아요,해시태그,링크
0,2019/06/14 12:34,06,14,soyulmom.jini,26,"[초곡야시장, 초곡장, 여기오늘, repost]",https://www.instagram.com/p/ByrN17zF7aL/
1,2019/06/14 11:02,06,14,dabeans_tofu,18,"[여기오늘, 초곡장, 초곡야시장]",https://www.instagram.com/p/ByrDXWfF-Fm/
2,2019/01/07 10:40,01,07,hello_7785,73,"[여행, 💙, 베트남, 다낭, 여행에미치다, 어딘가에서, 너무덥다, 추억, 여기오늘...",https://www.instagram.com/p/BsULRzVAu_t/
3,2018/12/05 22:46,12,05,rejinawang,33,"[여기오늘, 사랑하오여보, 와인무제한, 잘살아보세, 불편하게먹기, weddingan...",https://www.instagram.com/p/BrAgIOSFzfE/
4,2018/10/30 21:11,10,30,shin_julia_,24,"[인스타굿, 하라쇼, 좋은날, 감성스타그램, 🍁, 가을아안녕🍁, 여기오늘, 일상, ...",https://www.instagram.com/p/BpjoiIlnBGA/
5,2018/10/27 16:41,10,27,hee_t_t,37,"[불태우는날, 무섭다, 아들, 경험, 육아맘, 이뻐, 토요일, 행사, 서울로고, 잘...",https://www.instagram.com/p/BpbbWnug4iC/
6,2018/10/27 16:39,10,27,hee_t_t,28,"[ㅋㅋ, 행사, 한번이상갈려고노력, 서울행사, 와봄, 시청은처음, 장난감, 아들, ...",https://www.instagram.com/p/BpbbEc2An7Q/
7,2018/10/27 15:01,10,27,hee_t_t,31,[],https://www.instagram.com/p/BpbPslLAC0G/
8,2018/06/10 12:26,06,10,seo_doo_young,153,"[여기오늘, 아점, 쿠우쿠우, 폭식주의, 휴일스타그램]",https://www.instagram.com/p/Bj1DofaAEHc/
9,2018/03/10 01:19,03,10,doong0402,17,[],https://www.instagram.com/p/BgG93lGgyYS/


In [17]:
len(post_link)

32

In [19]:
post_linkx = post_link[8:]
len(post_linkx)

24

In [ ]:
작동여부 확인

In [5]:
#접속주소 설정
url = 'http://www.instagram.com'
path = 'C:\\Users\\rectworks\\Downloads\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(path)

ID = 'hanshow113'
PW = 'wlsldjtm12!'

instagram_login(ID, PW)

In [9]:
main_search('#여기오늘')

In [7]:
scroll_page()

In [23]:
id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []

date_based_crawling('2017.10.01')

In [24]:
insta_dict = {'날짜':date_list,
              '월':month_list,
              '일':day_list,
              '계정':id_list,
             '좋아요':like_list,
             '해시태그':tag_list,

             '링크':link_list}
df = pd.DataFrame(insta_dict)
df

,날짜,월,일,계정,좋아요,해시태그,링크
0,2020/03/18 01:00,03,18,untersbergbahn,26,[],https://www.instagram.com/p/B91zZ4BnIk0/
1,2020/03/06 15:09,03,06,cc_coopcenter,13,"[여기오늘, 춘천시협동조합지원센터, www, 홈페이지, 네이쳐앤드피플]",https://www.instagram.com/p/B9YbBjupqfI/
2,2019/07/26 00:04,07,26,eun_kyung_sin,23,"[비주얼, 게스트하우스, 애기데리고, 동네예쁜집, 편안해질, 여기오늘, 태양이장난감...",https://www.instagram.com/p/B0WBZ2wnkoG/
3,2019/06/14 12:34,06,14,soyulmom.jini,26,"[초곡야시장, repost, 초곡장, 여기오늘]",https://www.instagram.com/p/ByrN17zF7aL/
4,2019/06/14 11:02,06,14,dabeans_tofu,18,"[초곡장, 초곡야시장, 여기오늘]",https://www.instagram.com/p/ByrDXWfF-Fm/
5,2019/01/07 10:40,01,07,hello_7785,73,"[💙, 여행, 어딘가에서, 여행에미치다, 추억, 베트남, 여기오늘, 춥겠지, 너무덥...",https://www.instagram.com/p/BsULRzVAu_t/
6,2018/12/05 22:46,12,05,rejinawang,33,"[weddinganniversary, 사랑하오여보, 그림의떡😭, 여기오늘, 불편하게...",https://www.instagram.com/p/BrAgIOSFzfE/
7,2018/10/30 21:11,10,30,shin_julia_,24,"[여기오늘, 가을, 소확행, 🍁, 감성스타그램, 좋은날, 시월의어느멋진날에, 오늘여...",https://www.instagram.com/p/BpjoiIlnBGA/
8,2018/10/27 16:41,10,27,hee_t_t,37,"[육아맘, 무섭다, 행사, 잘해보자아들, 처음해보는, 서울로고, ㅠㅠ, 시청은처음,...",https://www.instagram.com/p/BpbbWnug4iC/
9,2018/10/27 16:39,10,27,hee_t_t,28,"[육아소통, 서울로고, 잘해보자아들, 3세아들, 한번이상갈려고노력, 넘쳐나는, 불태...",https://www.instagram.com/p/BpbbEc2An7Q/
